In [4]:
import pandas as pd

# Load the dataset
file_path = 'store_data.csv'
df = pd.read_csv(file_path)

# Display the first few rows to inspect the dataset
df.head()


,date,store_no,region,city,postal,street,longitude,latitude,store_area,location_type,...,footfall,avg_temperature,min_temperature,max_temperature,precipitation_mm,snow_depth_mm,wind_direction_degrees,wind_speed_kmh,peak_wind_gust_kmh,air_pressure_hpa
0,2021-02-12,2335,Bayern,Lauf,91207,Marktplatz 15,11.281178,49.511979,449.0,Fußgängerzone,...,NaN,-10.9,-18.0,-2.8,0.0,130.0,79.0,14.2,35.3,1033.1
1,2021-03-01,197,Rheinland-pfalz,Bitburg,54634,Trierer Str.19,6.523312,49.970275,263.0,Hauptgeschaeftsstrasse,...,NaN,6.6,0.5,14.0,0.0,NaN,73.0,13.1,NaN,1032.6
2,2021-03-01,441,Rheinland-pfalz,Annweiler,76855,Hauptstr. 35,7.963420,49.202622,350.0,Hauptgeschaeftsstrasse,...,NaN,7.4,2.4,12.8,0.0,0.0,45.0,12.5,NaN,1032.5
3,2021-03-01,632,Saarland,Lebach,66822,Am Markt 6,6.906569,49.411052,363.0,Fußgängerzone,...,3.0,5.8,0.6,12.1,0.0,0.0,74.0,13.9,41.4,1032.0
4,2021-03-01,1218,Rheinland-pfalz,Traben-Trarbach,56841,Poststr. 1 A,7.116635,49.950358,354.0,Hauptgeschaeftsstrasse,...,3.0,5.5,0.3,12.4,0.0,0.0,51.0,11.2,40.3,1033.0


In [5]:
# Get a summary of the dataset
df.info()

# Check for missing values
missing_values = df.isnull().sum()

# Display missing values
print("Missing values in each column:")
print(missing_values)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 414645 entries, 0 to 414644
Data columns (total 23 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   date                    414645 non-null  object 
 1   store_no                414645 non-null  int64  
 2   region                  414645 non-null  object 
 3   city                    414645 non-null  object 
 4   postal                  414645 non-null  int64  
 5   street                  414645 non-null  object 
 6   longitude               414645 non-null  float64
 7   latitude                414645 non-null  float64
 8   store_area              414645 non-null  float64
 9   location_type           414645 non-null  object 
 10  turnover                414645 non-null  float64
 11  population              414645 non-null  int64  
 12  competitor_count        414645 non-null  int64  
 13  footfall                245532 non-null  float64
 14  avg_temperature     

In [7]:
# Remove rows with missing values in 'footfall' column
df_cleaned = df.dropna(subset=['footfall'])

# Confirm the dataset size after cleaning
print(f"Dataset size after cleaning: {df_cleaned.shape}")

# Handle categorical column for location ('city') 
# Group cities with low frequencies into an 'Other' category
top_cities = df_cleaned['city'].value_counts().nlargest(50).index  # Top 50 cities
df_cleaned['city'] = df_cleaned['city'].apply(lambda x: x if x in top_cities else 'Other')


Dataset size after cleaning: (245532, 23)


C:\Users\amrit\AppData\Local\Temp\ipykernel_20628\1069177521.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['city'] = df_cleaned['city'].apply(lambda x: x if x in top_cities else 'Other')


In [8]:
# Define features and target
selected_features = ['store_area', 'footfall', 'avg_temperature', 'city', 
                     'precipitation_mm', 'air_pressure_hpa', 'wind_speed_kmh']
X = df_cleaned[selected_features]
y = df_cleaned['turnover']


In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

# Define preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='mean'), ['store_area', 'footfall', 'avg_temperature', 
                                                 'precipitation_mm', 'air_pressure_hpa', 
                                                 'wind_speed_kmh']),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=True), ['city'])  # Updated parameter
    ]
)


In [10]:
from sklearn.pipeline import Pipeline
import xgboost as xgb

# Define the XGBoost model
xgboost_model = xgb.XGBRegressor(random_state=42, n_estimators=100, learning_rate=0.1, max_depth=6)

# Create the complete pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', xgboost_model)
])


In [11]:
from sklearn.model_selection import train_test_split

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the pipeline
pipeline.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', SimpleImputer(),
                                                  ['store_area', 'footfall',
                                                   'avg_temperature',
                                                   'precipitation_mm',
                                                   'air_pressure_hpa',
                                                   'wind_speed_kmh']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['city'])])),
                ('model',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynod...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.1,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=6, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=100, n_jobs=None,
                              num_parallel_tree=None, random_state=42, ...))])

In [12]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Make predictions on the test set
y_pred = pipeline.predict(X_test)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

# Print evaluation metrics
print(f"XGBoost Model Performance:\nMAE: {mae:.2f}\nRMSE: {rmse:.2f}\nR² (Accuracy): {r2:.2f}")


XGBoost Model Performance:
MAE: 30.90
RMSE: 41.47
R² (Accuracy): 0.44
